# 自定义层

构造一个没有任何参数的自定义层

In [1]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

将层作为组件合并到更复杂的模型中

In [2]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(2.7940e-09, grad_fn=<MeanBackward0>)

带参数的层

In [3]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 1.9832,  0.9289,  1.3470],
        [-1.1857, -0.9701,  0.0247],
        [-1.4844,  0.3752,  2.1760],
        [-0.7389, -0.4874, -0.7060],
        [ 1.4744, -1.3758,  0.6738]], requires_grad=True)

使用自定义层直接执行前向传播计算

In [4]:
linear(torch.rand(2, 5))

tensor([[0.5338, 0.0880, 2.1402],
        [0.0000, 0.0000, 0.3480]])

使用自定义层构建模型

In [5]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[14.3284],
        [10.8282]])

## 练习

1. 设计一个接受输入并计算张量降维的层，它返回 $y_k=\sum_{i,j} W_{ijk}x_ix_j$。

In [21]:
class ReductionLayer(nn.Module):
  def __init__(self, in_units, units):
    super().__init__()
    self.weight = nn.Parameter(torch.randn((in_units, in_units, units)))
    
  def forward(self, X):
    return X @ self.weight.transpose(0, -1) @ X

In [22]:
net = ReductionLayer(64, 4)
net(torch.rand(64))

tensor([  0.9289, -27.8902, -32.8991, -28.9719], grad_fn=<MvBackward0>)

2. 设计一个返回输入数据的傅立叶系数前半部分的层。

In [8]:
import numpy as np

# 生成一些示例数据，比如一个正弦波
fs = 1000  # 采样率
t = torch.arange(0, 1, 1/fs)  # 时间数组
f = 5  # 正弦波频率
signal = torch.sin(2 * np.pi * f * t)

In [10]:
signal.shape

torch.Size([1000])

In [17]:
torch.fft.fft(signal).shape

torch.Size([1000])

In [27]:
class HalfFFTLayer(nn.Module):
  def __init__(self):
    super().__init__()
    
  def forward(self, X):
    half_len = X.shape[-1] // 2
    return torch.fft.fft(X, dim=-1)[..., :half_len]

In [33]:
net = HalfFFTLayer()
net(signal).shape, net(torch.rand((64,32,16))).shape

(torch.Size([500]), torch.Size([64, 32, 8]))